# Traffic Sign Identification using Deep Neural Networks

## Description
Lorem Ipsum

## Setup and Image Preprocessing

### Imports

In [1]:
import cv2
import pickle
import csv
from skimage import exposure

#A tua mina, tem ganda vagina
#Mas não foste tu quem escavou bro, foi o Chico da Tina

### Preprocessing Function
Resizes and Normalizes all pictures in all of our datasets and then stores them in a dictionary.

In [2]:
def preprocess_images(file_name):
    path = './Dataset/'   # Change this for the path corresponding to your base Dataset directory

    img_infos = {'labels':[],'images':[]}

    with open(path+file_name,"r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',', quotechar='|')
        next(reader)
        
        for row in reader:

            img = cv2.imread(path+row[-1], cv2.IMREAD_UNCHANGED)    
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Convert image to Grayscale
            
            img = cv2.resize(img, (32,32),interpolation = cv2.INTER_AREA) #Resize the image to 32x32

            #img = cv2.addWeighted( img, 1.2, img, 0, 0) #Increase contrast
            
            cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX) #Normalize
            
            img_eq = exposure.equalize_adapthist(img, clip_limit=0.03)
            
            img_infos['images'].append(img_eq) #Save Image pixels
            img_infos['labels'].append(row[-2]) #Save image Label
            
    return img_infos

### Save treated data as Pickle
Srsly, funniest thing I've ever seen

In [4]:
img_infos = preprocess_images("Test.csv")
with open("./Dataset/PickledData/Test.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)
    
print("> Finished writing tests")
img_infos = preprocess_images("Train.csv")
with open("./Dataset/PickledData/Train.p", 'wb') as pickle_rick:
    pickle.dump(img_infos,pickle_rick)
print("> Finished writing train")


finished writing tests
finished writing train


## Neural Network

### Step 0 - Imports

In [1]:
import random
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle

ModuleNotFoundError: No module named 'google.protobuf'

### Step 1 - Get the data into the sets

In [5]:
with open("./Dataset/PickledData/Test.p", "rb") as f:
    test = pickle.load(f)
with open("./Dataset/PickledData/Train.p", "rb") as f:
    train = pickle.load(f)

# Get the training set
x_train, y_train = train['images'], train['labels']

# Get the cross validation set
x_cv = []
y_cv = []
for i in range(int(0.2*len(x_train))): # Randomly move 20% of the training examples into the cross-validation set
    index = random.randint(0, len(x_train)-1)
    x_cv.append(x_train.pop(index))
    y_cv.append(y_train.pop(index))
    
# Get the test set
x_test, y_test = test['images'], test['labels']

# Convert all our sets into numPy Arrays
x_train = np.array(x_train).reshape((len(x_train), 32, 32, 1))
y_train = keras.utils.to_categorical(np.array(y_train))

x_cv = np.array(x_cv).reshape((len(x_cv), 32, 32, 1))
y_cv = keras.utils.to_categorical(np.array(y_cv))

x_test = np.array(x_test).reshape((len(x_test), 32, 32, 1))
y_test = keras.utils.to_categorical(np.array(y_test))

### Step 2 - Treat the data (Normalize and Shuffle)

In [6]:
# Shuffle the training set
x_train, y_train = shuffle(x_train, y_train)

### Step 3 - Define constants

In [7]:
EPOCHS = 50
BATCH_SIZE = 200

LEARNING_RATE = 0.002 # Define the learning rate to be used by our optimizer


### Step 4 - Define the Neural Network

In [8]:
def LeNetModel(no_output=6, rgb=True, dropout=0):
    input_shape = (32,32,3) if rgb else (32,32,1) # Define the image's size
    
    model = keras.Sequential() # Define the Keras Model
    
    # First Layer - Convolutional ; Input = 32x32x3 for RGB , 32x32x1 for GrayScale ; Output = 28x28x6
    model.add(keras.layers.Conv2D(
                filters=no_output, kernel_size=(3, 3),
                activation='relu', input_shape=input_shape
                )
             )
    
    # Second Layer - Subsampling (Average Pooling) ; Input = 28x28x6 ; Output = 14x14x6
    model.add(keras.layers.AveragePooling2D())
    
    # Third Layer - Convolutional ; Input = 14x14x6 ; Output = 10x10x16
    model.add(keras.layers.Conv2D(
                    filters=16, kernel_size=(3, 3),
                    activation='relu'
                    )
                 )
    
    # Fourth Layer - Subsampling (Average Pooling) ; Input = 10x10x16 ; Output = 5x5x16
    model.add(keras.layers.AveragePooling2D())
    
    model.add(keras.layers.Flatten()) # Flatten the last layer's output to pass it to the Fully Connected layers
    
    # Fifth Layer - Fully Connected ; Input = 5x5x16 ; Output = 120x1
    model.add(keras.layers.Dense(units=120, activation='relu'))

    # Sixth Layer - Fully Connected ; Input = 120x1 ; Output = 84x1
    model.add(keras.layers.Dense(units=84, activation='relu'))
    
    model.add(keras.layers.Dropout(dropout))
    
    # Output Layer - Output = 43x1
    model.add(keras.layers.Dense(units=43, activation = 'softmax'))
    
    return model

### Step 5 - Create the model and define evaluation

In [12]:
model = LeNetModel(rgb=False, dropout=0.50)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

### Step 6 - Insert the training set

In [13]:
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(x_cv, y_cv))

Train on 31368 samples, validate on 7841 samples
Epoch 1/50
31368/31368 [==============================] - 5s 164us/sample - loss: 2.5409 - accuracy: 0.3054 - precision_1: 0.8058 - recall_1: 0.1336 - val_loss: 1.1143 - val_accuracy: 0.6736 - val_precision_1: 0.9018 - val_recall_1: 0.4286
Epoch 2/50
31368/31368 [==============================] - 4s 135us/sample - loss: 1.0547 - accuracy: 0.6658 - precision_1: 0.8444 - recall_1: 0.5149 - val_loss: 0.6046 - val_accuracy: 0.8186 - val_precision_1: 0.9124 - val_recall_1: 0.7224
Epoch 3/50
31368/31368 [==============================] - 4s 135us/sample - loss: 0.7001 - accuracy: 0.7744 - precision_1: 0.8731 - recall_1: 0.6806 - val_loss: 0.4035 - val_accuracy: 0.8865 - val_precision_1: 0.9304 - val_recall_1: 0.8373
Epoch 4/50
31368/31368 [==============================] - 4s 133us/sample - loss: 0.5350 - accuracy: 0.8287 - precision_1: 0.8938 - recall_1: 0.7657 - val_loss: 0.3144 - val_accuracy: 0.9056 - val_precision_1: 0.9378 - val_recall_1

Epoch 35/50
31368/31368 [==============================] - 4s 134us/sample - loss: 0.0369 - accuracy: 0.9873 - precision_1: 0.9897 - recall_1: 0.9858 - val_loss: 0.0668 - val_accuracy: 0.9865 - val_precision_1: 0.9875 - val_recall_1: 0.9860
Epoch 36/50
31368/31368 [==============================] - 4s 133us/sample - loss: 0.0329 - accuracy: 0.9886 - precision_1: 0.9899 - recall_1: 0.9873 - val_loss: 0.0698 - val_accuracy: 0.9844 - val_precision_1: 0.9857 - val_recall_1: 0.9842
Epoch 37/50
31368/31368 [==============================] - 4s 134us/sample - loss: 0.0402 - accuracy: 0.9859 - precision_1: 0.9877 - recall_1: 0.9841 - val_loss: 0.0677 - val_accuracy: 0.9843 - val_precision_1: 0.9861 - val_recall_1: 0.9837
Epoch 38/50
31368/31368 [==============================] - 4s 127us/sample - loss: 0.0329 - accuracy: 0.9881 - precision_1: 0.9897 - recall_1: 0.9865 - val_loss: 0.0714 - val_accuracy: 0.9838 - val_precision_1: 0.9848 - val_recall_1: 0.9834
Epoch 39/50
31368/31368 [===========

In [14]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test precision:', score[2])
print('Test recall:', score[3])

12630/12630 [==============================] - 1s 97us/sample - loss: 0.3738 - accuracy: 0.9451 - precision_1: 0.9485 - recall_1: 0.9444
Test loss: 0.37375960503714295
Test accuracy: 0.9450515
Test precision: 0.94854873
Test recall: 0.9444181


## Discussion of Results


### RGB vs Grayscale

### Epoch Numbers

### Data Augmentation

### Other Preprocessing Parameter changes (Contrast, lighting, etc)

### Dropout